In [ ]:
import numpy as np
import cupy as cp
import sys
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from types import SimpleNamespace
import h5py
import dxchange
import warnings
import pandas as pd
warnings.filterwarnings("ignore", message=f".*peer.*")

sys.path.insert(0, '..')
from utils import *
from rec import Rec

In [ ]:
cf = 0
ntheta = 1
path = f'/data/vnikitin/ESRF/ID16A/20240924_rec_ca/data/'
iter = 2048
with  h5py.File(f'{path}/data_atomium.h5','r') as fid:
    code = fid[f'/exchange/code{iter}'][:].astype('complex64')
    q = fid[f'/exchange/prb{iter}'][:].astype('complex64')    
    shifts_cor = fid[f'/exchange/shifts_cor{iter}'][::7200//ntheta].astype('float32')
    shifts_code = fid[f'/exchange/shifts_code{iter}'][::7200//ntheta].astype('float32')    
    z1 = fid['/exchange/z1'][0]
    detector_pixelsize = fid['/exchange/detector_pixelsize'][0]
    focusdetectordistance = fid['/exchange/focusdetectordistance'][0]
    energy = fid['/exchange/energy'][0]

with  h5py.File(f'{path}/data_ca.h5','r') as fid:
    z1c = fid['/exchange/z1'][0] 

q = q[q.shape[0]//2-q.shape[0]//2**cf//2:q.shape[0]//2+q.shape[0]//2**cf//2,
        q.shape[1]//2-q.shape[1]//2**cf//2:q.shape[1]//2+q.shape[1]//2**cf//2]
code = code[code.shape[0]//2-code.shape[0]//2**cf//2:code.shape[0]//2+code.shape[0]//2**cf//2,
        code.shape[1]//2-code.shape[1]//2**cf//2:code.shape[1]//2+code.shape[1]//2**cf//2]
    
shifts_cor/=2**cf
shifts_code/=2**cf

In [ ]:
wavelength = 1.24e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
z2 = focusToDetectorDistance-z1
distance = (z1*z2)/focusToDetectorDistance
magnification = focusToDetectorDistance/z1
voxelsize = np.abs(detector_pixelsize/magnification) # object voxel size
magnifications2 = z1/z1c
distancec = (z1-z1c)/(z1c/z1)
show = True

In [ ]:
args = SimpleNamespace()
args.ngpus = 4

args.n = 2048//2**cf
args.ntheta = ntheta
args.ncode = 8192*args.n//2048
args.pad = 32
args.npsi = args.n + 2 * args.pad
args.nq = args.n + 2 * args.pad
args.ex = 8
args.npatch = args.nq + 2 * args.ex
args.nchunk = 32

args.voxelsize = voxelsize
args.wavelength = wavelength
args.distance = distance
args.distancec = distancec
args.rotation_axis=756*args.n/1024-args.n//4+args.pad
args.show = True
args.theta = np.linspace(0,np.pi,ntheta).astype('float32')
# create class
# cl_rec = Rec(args)
print(voxelsize,distance,distancec)

In [ ]:
q = np.pad(q,((args.pad,args.pad),(args.pad,args.pad)),'symmetric')

# introdcue some floating part in shifts code:

In [ ]:
np.random.seed(10)
shifts_code+=2*(np.random.random([ntheta,2])-0.5).astype('float32')

In [ ]:
mshow_polar(q,show)

In [ ]:
mshow_complex(q,show)